<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [138]:
import os
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
from snowballstemmer import RussianStemmer, EnglishStemmer
import pymorphy2
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import json

In [141]:
data_dir = "/data/share/lab05data/"
# Индексы, указанные в личном кабинете
ids = set([2561, 514, 2570, 3085, 1040, 1046, 24, 3610, 3612, 859, 33, 945, 2087, 1885, 1580, 1074,
           3639, 1593, 2485, 573, 63, 3649, 3139, 582, 2049, 1608, 1098, 1099, 2639, 3668, 1626, 607, 
           2656, 1121, 1635, 3178, 1644, 2158, 2159, 2238, 3699, 3700, 2682, 2173, 2174, 3203, 2692,
           2184, 2187, 536, 3213, 3214, 3728, 3729, 658, 2195, 3222, 2199, 2200, 624, 672, 161, 1698,
           3748, 711, 3239, 3244, 1710, 176, 3763, 1396, 1717, 695, 698, 1723, 1725, 702, 1215, 1217,
           1320, 3267, 196, 710, 3783, 3272, 715, 2255, 724, 1758, 3808, 3300, 3818, 748, 3822, 751,
           2288, 3180, 3317, 2806, 3831, 384, 623, 3030, 3842, 3850, 2316, 783, 2324, 1305, 2332, 1311, 
           1825, 292, 1831, 562, 812, 2863, 3376, 307, 820, 3551, 2361, 833, 2374, 2377, 1355, 3920,
           337, 2386, 1367, 1308, 347, 861, 352, 3473, 3940, 870, 2921, 2924, 1524, 13, 2758, 372,
           1909, 887, 888, 382, 1174, 897, 3137, 903, 1419, 579, 1006, 1937, 917, 406, 2456, 414, 2463,
           3489, 668, 2981, 429, 2760, 1457, 3508, 3509, 1439, 2634, 3001, 3516, 445, 2379, 2496, 1697,
           971, 972, 2510, 464, 2003, 1956, 3542, 2521, 2010, 1500, 2526, 479, 3041, 482, 2532, 485, 
           2023, 3564, 3566, 766, 2548, 3576, 3066, 1020])

Круто, что разделил на английские и русские слова, это было моим запасным вариантом, если бы чекер не засчитал задание :)

In [107]:
# Стеммеры для русских и английских слов
rs = RussianStemmer()
es = EnglishStemmer()
# Лемматизаторы для русских и английских слов
morph = pymorphy2.MorphAnalyzer()
lemmatizer = WordNetLemmatizer()
# Русские и английские стоп-слова
stops_eng = set(stopwords.words("english"))
stops_rus = set(stopwords.words("russian"))
stops = stops_eng.union(stops_rus)

# Функция определения английского слова
def is_english(word):
    if re.match("[a-z]", word) is not None:
        return True
    else:
        return False

# Функция определения русского слова
def is_russian(word):
    if re.match("[а-я]", word) is not None:
        return True
    else:
        return False

# Функция получения нормальной формы слова
def get_normal_form(word):
    # для английских слов
    if is_english(word):
        return lemmatizer.lemmatize(word)
    # для русских слов
    if is_russian(word):
        return morph.parse(word)[0].normal_form
    return ""

# Функция получения стеммированной формы слова
def stem_word(word):
    # для английских слов
    if is_english(word):
        return es.stemWord(word)
    # для русских слов
    if is_russian(word):
        return rs.stemWord(word)
    return ""

# Функция получения из текста файла набора (преобразованных) слов
def text_to_wordlist( text, remove_stopwords=False, stemming=False, lemming=False ):
    '''
    Функция получения из текста файла набора (преобразованных) слов
    
    text: текст, str
    remove_stopwords: флаг удаления стоп-слов, bool
    stemming: флаг стемминга слов, bool
    lemming: флаг лемматизации слов, bool
    '''
    # Достаем текст из файла, игнорируем тэги
    text = BeautifulSoup(text).get_text()
    # Оставляем только латинские и русские буквы
    text = re.sub("[^a-zA-Zа-яА-Я]"," ", text)
    words = text.lower().split()
    # удаление стоп-слов
    if remove_stopwords:
        words = [w for w in words if not w in stops]
    # стемминг
    if stemming:
        words = [stem_word(word) for word in words]
    # лемматизация
    if lemming:
        words = [get_normal_form(word) for word in words]

    return(' '.join(words))

По-хорошему надо файл закрывать после открытия. В целом, на таком количестве это некритично, но правила хорошего тона и тд. Еще, как мне кажется, проще было использовать датафреймы, чтобы потом не писать циклы, но это уже дело вкуса.

In [106]:
# массив базовых текстов
base_texts = []
# словарь тестовых текстов (id -> text)
test_texts = dict()
# проход по файлам в указанной папке
for root, dirs, files in os.walk(data_dir):
    for file in files:
        file_path = os.path.join(root, file)
        text = open(file_path).read()
        # получение набора слов из файла (удаление стоп-слов и лемматизация)
        wordlist = text_to_wordlist(text, remove_stopwords=True, stemming=False, lemming=True)
        if "base" in file:
            base_texts.append( wordlist )
        if "test" in file:
            # получение id текста из имени файла
            text_id = int(file.split('.')[0].split('_')[1])
            if text_id in ids:
                test_texts[text_id] = wordlist

/opt/anaconda/envs/bd9/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/anaconda/envs/bd9/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [108]:
vectorizer = CountVectorizer(analyzer = "word",   
                             tokenizer = None,    
                             preprocessor = None, 
                             stop_words = None,   
#                              max_features = 5000
                            ) 

Не очень понятно, зачем присваивание all_texts = base_texts. И по-моему так же, как у меня - тут используется fit_transform, а потом опять transform. Можно либо тут оставить только fit, либо сразу делать fit_transform

In [117]:
# Обучаем CountVectorizer на базовых текстах
all_texts = base_texts
vectorizer.fit_transform(all_texts)

<20x1054 sparse matrix of type '<class 'numpy.int64'>'
	with 2541 stored elements in Compressed Sparse Row format>

In [120]:
# Получаем векторное представление базовых и тестовых текстов
base_features = vectorizer.transform(base_texts)
test_features = vectorizer.transform([test_texts[id_] for id_ in ids])

In [129]:
# Считаем косинусную близость между текстами и суммируем полученные близости для каждого из тестового текста
similarities = cosine_similarity(base_features, test_features).sum(axis=0)
# Находим среднюю сумму косинусных близостей
mean_s = similarities.mean()

In [136]:
# Определяем принадлежность тестового текста к заданной тематике, сравнивая с пороговым значением
result = {"defined": [], "other": []}
for i, id_ in enumerate(ids):
    if similarities[i] > mean_s:
        result["defined"].append(id_)
    else:
        result["other"].append(id_)

In [140]:
# Вывод результата в файл
with open("lab05.json", "w") as f:
    f.write( json.dumps(result) )